### Reranking experiment

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Document, Prefetch, FusionQuery, VectorParams, PayloadSchemaType
from qdrant_client.models import PointStruct, FusionQuery, SparseVectorParams

import pandas as pd
import openai
import cohere

In [ ]:
from qdrant_client import QdrantClient
import openai
import instructor
instructor_prompt = instructor.from_openai(openai.OpenAI())

def create_embeddings(text, model="text-embedding-3-small"):
   
    response = openai.embeddings.create(
        model=model,
        input=text
    )
        
    return response.data[0].embedding

def retrieve_embedding_data(qd_client: QdrantClient, query, collection_name, k=5):
    querry_embeddings = create_embeddings(query)
    response = qd_client.query_points(
        collection_name=collection_name,
        prefetch=[Prefetch(
            query=querry_embeddings,
            using="text-embedding-3-small",
            limit=20),
            Prefetch(
                query=Document(text=query, model="qdrant/bm25"),
                using="bm25",
                limit=20)
            ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )
    retrieved_context_ids = []
    retrieved_context = []
    retrieved_scores = []
    retrieved_context_ratings = []
    
    for point in response.points:
        retrieved_context_ids.append(point.payload["parent_asin"])
        retrieved_context.append(point.payload["description"])
        retrieved_scores.append(point.score)
        retrieved_context_ratings.append(point.payload["average_rating"])

    # return dictionary of retrieved data
    return {
        "context_ids": retrieved_context_ids,
        "context": retrieved_context,
        "scores": retrieved_scores,
        "context_ratings": retrieved_context_ratings
    }

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")
sample_query = "top headphones under 1000$"
retrieved_data = retrieve_embedding_data(qdrant_client, sample_query, "amazon_items-collection-hybrid-02", 20)
retrieved_data

### Re-ranking 

In [ ]:
import os
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
cohere_client = cohere.ClientV2()

retrieved_context_for_rerank = retrieved_data["context"]
retrieved_context_for_rerank

In [ ]:
reranked_context = cohere_client.rerank(
    model="rerank-v4.0-fast",
    query=sample_query,
    documents=retrieved_context_for_rerank,
    top_n=20,
)
reranked_context

In [ ]:
reranked_results = [retrieved_context_for_rerank[result.index] for result in reranked_context.results]


In [ ]:
reranked_results